<a href="https://colab.research.google.com/github/rawoofcryptos/premiumanalyser/blob/main/Option_Discount_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stream Option Data

In [ ]:
#@title { form-width: "25%", display-mode: "form" }
display(Javascript("""google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})"""))
STREAM_DATA = True #@param { type: 'boolean' }
if STREAM_DATA:
  if INCLUDE_NIFTY or INCLUDE_BANKNIFTY or INCLUDE_FINNIFTY:
    try:
      option_chain = FetchOptionData(
        CREDS, EMAIL, 
        PASSWORD_5PAISA, DOB_YYYYMMDD, 
        BNF_NIFTY_EXPIRY_, FINNIFTY_EXPIRY_,
        INCLUDE_NIFTY, INCLUDE_BANKNIFTY, INCLUDE_FINNIFTY,
        BNF_NIFTY_FUT_EXPIRY_, FINNIFTY_FUT_EXPIRY_)
      
      option_chain.stream()

    except (FetchExpiryException, InvalidLoginException, 
            InvalidLoginCredentialsException, InvalidFutureExpiryDateException,
            InvalidOptionExpiryDateException, OptionChainFetchException,
            FuturesFetchException, SpotFetchException, 
            OptionChainFetchException) as e:
      pass
    except KeyboardInterrupt: 
      clear_output(wait=True)
      display(HTML("<h2 style='color: #FF4500'>Error : Option Chain fetch interrupted</h2>"))
  else:
    display(HTML(f"<h2 style='color: #FD7F20'>Select atleast one index</h2>"))

# Provide Index Details

In [6]:
#@title { form-width: "500%", display-mode: "form" }

INCLUDE_NIFTY = True #@param { type: 'boolean' }
INCLUDE_BANKNIFTY = True #@param { type: 'boolean' }
INCLUDE_FINNIFTY = False #@param { type: 'boolean' }
#@markdown __________________________________

BNF_NIFTY_FUT_EXPIRY = "2023-03-29" #@param { type: 'date' }
FINNIFTY_FUT_EXPIRY = "2023-03-28" #@param { type: 'date' }
#@markdown __________________________________

BNF_NIFTY_FUT_EXPIRY_ = convertTimeString(BNF_NIFTY_FUT_EXPIRY)
FINNIFTY_FUT_EXPIRY_ = convertTimeString(FINNIFTY_FUT_EXPIRY)

BNF_NIFTY_EXPIRY = "2023-03-16" #@param { type: 'date' }
FINNIFTY_EXPIRY = "2023-03-21" #@param { type: 'date' }

BNF_NIFTY_EXPIRY_ = getEpochTime(BNF_NIFTY_EXPIRY)
FINNIFTY_EXPIRY_ = getEpochTime(FINNIFTY_EXPIRY)

# Provide 5Paisa Credentials

In [2]:
#@title { form-width: "500%", display-mode: "form" }
import os
import sys
import subprocess
from IPython.display import display, HTML, Javascript, clear_output
from google.colab import files
sys.tracebacklimit = 0

file_name = 'creds.txt' #@param { type: 'string'}

if not os.path.exists('/content/creds.txt'):
  display(HTML("<h2 style='color: #FD7F20'>Add Credenitals</h2>"))
  files.upload()
  clear_output(wait=True)
  with open(file_name, 'r') as f:
    creds = f.read().lstrip().rstrip().split('\n')

    creds = [c.split('=')[1].strip() for c in creds]

    EMAIL = creds[0]
    PASSWORD_5PAISA = creds[1]
    DOB_YYYYMMDD = creds[2]

    APP_NAME = creds[3]
    APP_SOURCE = creds[4]
    USER_ID = creds[5]
    PASSWORD = creds[6]
    USER_KEY = creds[7]
    ENCRYPTION_KEY = creds[8]

    CREDS = {'APP_NAME':APP_NAME, 
             'APP_SOURCE':APP_SOURCE,
             'USER_ID': USER_ID,
             'PASSWORD': PASSWORD,
             'USER_KEY': USER_KEY,
             'ENCRYPTION_KEY': ENCRYPTION_KEY}
    display(HTML("<h2 style='color: #00D100'>Credentials Added</h2>"))


if not os.path.exists('/content/.config/discount-check'):
  os.chdir('/content/.config')
  display(HTML("<h2 style='color: #FD7F20'>Downloading 5Paisa...</h2>"))
  !git clone https://github.com/mhdSharuk/discount-check.git --quiet
  os.chdir('./discount-check')
  result = subprocess.run(["pip", "install", "-r", "./requirements.txt"], capture_output=False)
  display(HTML(f"<h2 style='color: #00D100'>Finished Downloading</h2>"))

from py5paisa import (
    FivePaisaClient,
    InvalidLoginCredentialsException,
    InvalidFutureExpiryDateException,
    InvalidLoginException,
    FetchExpiryException,
    InvalidOptionExpiryDateException,
    FuturesFetchException, 
    SpotFetchException, 
    OptionChainFetchException,
    getEpochTime,
    convertTimeString
    )

from discount_check import FetchOptionData